该输出解析器允许用户以流行的 XML 格式从 LLM 获取结果。

请记住，大型语言模型是有漏洞的抽象！您必须使用具有足够容量的 LLM 来生成格式正确的 XML。




In [2]:
from langchain.output_parsers import XMLOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAI

API_KEY="sk-9846f14a2104490b960adbf5c5b3b32e" #老师的Key
BASE_URL = "https://api.deepseek.com"
MODEL= "deepseek-chat"

chat = ChatOpenAI(
    model= MODEL,  # 默认的大模型为GPT-3.5-turbo，比较便宜
    openai_api_base= BASE_URL,
    openai_api_key= API_KEY,
    temperature=0.3,
)


In [3]:
#直接用提示词调用chat.invoke解决xml输出问题

actor_query = "生成汤姆·汉克斯的电影目录。"
output = chat.invoke(
    f"""{actor_query}

响应以xml的结构返回，使用如下xml结构
```
<xml>
<movie>电影1</movie>
<movie>电影2</movie>
<xml>
``` 
"""
)
print(output.content)

```xml
<xml>
<movie>阿甘正传</movie>
<movie>拯救大兵瑞恩</movie>
<movie>荒岛余生</movie>
<movie>费城故事</movie>
<movie>绿色奇迹</movie>
<movie>西雅图夜未眠</movie>
<movie>猫鼠游戏</movie>
<movie>达芬奇密码</movie>
<movie>萨利机长</movie>
<movie>玩具总动员系列</movie>
</xml>
```


In [4]:
#对chat.invoke的输出，调用Parser进行解析输出。
parser = XMLOutputParser()
parser.invoke(output)

{'xml': [{'movie': '阿甘正传'},
  {'movie': '拯救大兵瑞恩'},
  {'movie': '荒岛余生'},
  {'movie': '费城故事'},
  {'movie': '绿色奇迹'},
  {'movie': '西雅图夜未眠'},
  {'movie': '猫鼠游戏'},
  {'movie': '达芬奇密码'},
  {'movie': '萨利机长'},
  {'movie': '玩具总动员系列'}]}

In [10]:
#方式一，显式说明format_instructions，以便用中文显式
parser = XMLOutputParser()
format_instructions = """响应以xml的结构返回，使用如下xml结构
```
<xml>
<movie>电影1</movie>
<movie>电影2</movie>
<xml>
```
"""
format_instructions

'响应以xml的结构返回，使用如下xml结构\n```\n<xml>\n<movie>电影1</movie>\n<movie>电影2</movie>\n<xml>\n```\n'

In [11]:
query = "生成朱丽叶罗伯茨的电影目录。"
prompt = PromptTemplate(
    template="""{query}\n{format_instructions}""",
    input_variables=["query"],
    partial_variables={"format_instructions":format_instructions},
)

chain = prompt | chat | parser

output = chain.invoke({"query": query})
print(output)

{'xml': [{'movie': '漂亮女人'}, {'movie': '诺丁山'}, {'movie': '永不妥协'}, {'movie': '风月俏佳人'}, {'movie': '我最好朋友的婚礼'}, {'movie': '蒙娜丽莎的微笑'}, {'movie': '美食、祈祷和恋爱'}, {'movie': '奇迹男孩'}, {'movie': '八月：奥色治郡'}, {'movie': '钢木兰'}]}


In [8]:
#方式二，调用parser.get_format_instructions说明format_instructions，一般结果会是英文显式
parser = XMLOutputParser()
format_instructions =  parser.get_format_instructions()
format_instructions

'The output should be formatted as a XML file.\n1. Output should conform to the tags below.\n2. If tags are not given, make them on your own.\n3. Remember to always open and close all the tags.\n\nAs an example, for the tags ["foo", "bar", "baz"]:\n1. String "<foo>\n   <bar>\n      <baz></baz>\n   </bar>\n</foo>" is a well-formatted instance of the schema.\n2. String "<foo>\n   <bar>\n   </foo>" is a badly-formatted instance.\n3. String "<foo>\n   <tag>\n   </tag>\n</foo>" is a badly-formatted instance.\n\nHere are the output tags:\n```\nNone\n```'

In [9]:
query = "生成朱丽叶罗伯茨的电影目录。"
prompt = PromptTemplate(
    template="""{query}\n{format_instructions}""",
    input_variables=["query"],
    partial_variables={"format_instructions":format_instructions},
)

chain = prompt | chat | parser

output = chain.invoke({"query": query})
print(output)

{'Filmography': [{'Actor': [{'Name': 'Julia Roberts'}, {'Movies': [{'Movie': [{'Title': 'Pretty Woman'}, {'Year': '1990'}]}, {'Movie': [{'Title': 'Erin Brockovich'}, {'Year': '2000'}]}, {'Movie': [{'Title': 'Notting Hill'}, {'Year': '1999'}]}, {'Movie': [{'Title': "Ocean's Eleven"}, {'Year': '2001'}]}, {'Movie': [{'Title': "My Best Friend's Wedding"}, {'Year': '1997'}]}, {'Movie': [{'Title': 'Runaway Bride'}, {'Year': '1999'}]}, {'Movie': [{'Title': 'Steel Magnolias'}, {'Year': '1989'}]}, {'Movie': [{'Title': 'Eat Pray Love'}, {'Year': '2010'}]}, {'Movie': [{'Title': 'The Pelican Brief'}, {'Year': '1993'}]}, {'Movie': [{'Title': 'August: Osage County'}, {'Year': '2013'}]}]}]}]}
